# TODO

# LinearRegression from Scratch


다음 가상 데이터를 이용해 사과와 오렌지 수확량을 예측하는 선형회귀 모델을 정의한다. 


|온도(F)|강수량(mm)|습도(%)|사과생산량(ton)|오렌지생산량|
|-|-|-|-:|-:|
|73|67|43|56|70|
|91|88|64|81|101|
|87|134|58|119|133|
|102|43|37|22|37|
|69|96|70|103|119|

```
사과수확량  = w11 * 온도 + w12 * 강수량 + w13 * 습도 + b1
오렌지수확량 = w21 * 온도 + w22 * 강수량 + w23 *습도 + b2
```

- `온도`, `강수량`, `습도` 값이 **사과**와, **오렌지 수확량**에 어느정도 영향을 주는지 가중치를 찾는다.
    - 모델은 사과의 수확량, 오렌지의 수확량 **두개의 예측결과를 출력**해야 한다.
    - 사과에 대해 예측하기 위한 weight 3개와 오렌지에 대해 예측하기 위한 weight 3개 이렇게 두 묶음, 총 6개의 weight를 정의하고 학습을 통해 가장 적당한 값을 찾는다.

## step 1.
## Train Dataset
- Train data는 feature(input)와 target(output) 각각 2개의 행렬로 구성한다.
- Feature는 `온도, 강수량, 습도` 3개의 변수를 가진다.
- Target은 `사과수확량, 오렌지 수확량` 2개의 값이다.

In [27]:
# input/output 2차원리스트 생성하기기
# input: 생산환경 (temp, rainfall, humidity) - (5, 3)
environs = [[73, 67, 43],
			[91, 88, 64],
			[87, 134, 58],
			[102, 43, 37],
			[69, 96, 70]]

# output: 생산량 - (apples, oranges) - (5, 2)
apple_orange_output = [[56, 70],
					   [81, 101],
					   [119, 133],
					   [22, 37],
					   [103, 119]]

In [29]:
import torch
# Dataset을 torch.Tensor(dtype은 float32)로 생성 후 X,y의 shape 확인하기

# 1. Tensoer 생성
X = torch.tensor(environs, dtype=torch.float32)
y = torch.tensor(apple_orange_output, dtype=torch.float32)

# 2. shape 확인
X, y, X.dtype, X.shape, y.shape

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.],
         [102.,  43.,  37.],
         [ 69.,  96.,  70.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.],
         [ 22.,  37.],
         [103., 119.]]),
 torch.float32,
 torch.Size([5, 3]),
 torch.Size([5, 2]))

## step 2. weight/bias 정의하기

In [32]:
# random(torch.randn) 값을 이용해서 weight/bias 를 정의하기하고 size 확인인

# 1. weight, bias 정의(requires_grad=True)
weight = torch.randn(3,2, requires_grad=True)
bias = torch.rand(2, requires_grad=True)

# 2. weight, bias size 확인
weight.shape, weight.size(), bias.shape, bias.size()

(torch.Size([3, 2]), torch.Size([3, 2]), torch.Size([2]), torch.Size([2]))

## step3. 모델링

In [33]:
# 모델은 y = X * w + b 꼴, loss함수는 MSE로 정의

## 1. 모델 정의
def model(X):
    return X @ weight + bias

## 2. loss 함수(MSE)
def loss_fn(y_pred, y):
    return torch.mean((y_pred - y)**2)

# 3. epochs =5000, lr = 0.00001로 모델링 완성하기기
epochs = 5000
n = 0.00001

for epoch in range(epochs):

    # 1. 추론
    y_pred = model(X)
    
    # 2. loss 계산
    loss = loss_fn(y_pred, y)

    # 3. 파라미터 들의 gradient 계산
    loss.backward()

    # 4. 파라미터 업데이트
    weight.data = weight.data - n * weight.grad
    bias.data = bias.data - n * bias.grad
    # 5. gradient 초기화
    weight.grad = None
    bias.grad = None

    ## 500 epoch, 마지막 epoch에서 loss의 log를 출력
    if epoch % 500 == 0 or epoch == epochs-1:
        print(f"[{epoch+1:04d}/{epochs}] - {loss.item():.5f}")

[0001/5000] - 4695.69434
[0501/5000] - 2.37322
[1001/5000] - 1.12781
[1501/5000] - 0.76863
[2001/5000] - 0.62769
[2501/5000] - 0.57229
[3001/5000] - 0.55052
[3501/5000] - 0.54196
[4001/5000] - 0.53859
[4501/5000] - 0.53726
[5000/5000] - 0.53674


## step 4. 기존의 X와 new_x로 model 결과확인하기

In [47]:
# 1. 위 모델에 X로 다시 추론하여, 예측값 p와 정답값 y 비교하기

p = model(X)

print(f"예측값(p):\n", p)
print(f"정답값(y):\n", y)

# 2. 새로운 데이터 [68, 82, 56] 넣어 결과 확인(dtype은 float32 & .unsqueeze(dim=0))

new_x = [[68, 82, 56]]
new_x = torch.tensor(new_x, dtype=torch.float32)
model(new_x)


예측값(p):
 tensor([[ 57.2711,  70.4085],
        [ 82.1432, 100.5912],
        [118.6696, 132.9800],
        [ 21.0664,  37.0134],
        [101.9436, 119.1162]], grad_fn=<AddBackward0>)
정답값(y):
 tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


tensor([[80.9051, 95.6265]], grad_fn=<AddBackward0>)